In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   743k      0  0:00:01  0:00:01 --:--:--  743k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[1.2835e-03, 1.0441e-02, 2.3956e-04,  ..., 2.1445e-02, 4.8704e-02,
         1.3139e-02],
        [2.0557e-02, 1.0153e-02, 4.3309e-02,  ..., 2.4939e-02, 2.0275e-02,
         3.2060e-02],
        [1.8538e-02, 8.5980e-06, 4.0185e-02,  ..., 2.2580e-02, 3.6572e-02,
         3.1395e-02],
        ...,
        [4.6458e-02, 3.1853e-02, 2.2208e-02,  ..., 4.4432e-02, 2.5513e-02,
         2.6589e-03],
        [5.6583e-03, 2.9308e-02, 4.4330e-02,  ..., 2.4085e-02, 3.5451e-02,
         5.0994e-03],
        [3.7921e-02, 3.8065e-02, 1.1789e-02,  ..., 3.9095e-02, 2.7979e-02,
         4.7558e-02]])
item_factors.weight tensor([[0.0109, 0.0400, 0.0482,  ..., 0.0421, 0.0492, 0.0175],
        [0.0102, 0.0496, 0.0010,  ..., 0.0132, 0.0429, 0.0258],
        [0.0287, 0.0130, 0.0101,  ..., 0.0109, 0.0047, 0.0332],
        ...,
        [0.0411, 0.0221, 0.0466,  ..., 

In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()#
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

/home/kangjunekoo/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.0641437557143
iter #1 Loss: 4.7460717386400635
iter #2 Loss: 2.4750814465096758
iter #3 Loss: 1.7207717258615542
iter #4 Loss: 1.3451448724354584
iter #5 Loss: 1.1282905065620006
iter #6 Loss: 0.9914040203929553
iter #7 Loss: 0.9004793683738272
iter #8 Loss: 0.8371988979692991
iter #9 Loss: 0.7924038547322835
iter #10 Loss: 0.759204307485958
iter #11 Loss: 0.7350524402587547
iter #12 Loss: 0.7162840021443246
iter #13 Loss: 0.701357619373629
iter #14 Loss: 0.6902854187688247
iter #15 Loss: 0.6817879514948366
iter #16 Loss: 0.6751513560184368
iter #17 Loss: 0.6698768137372689
iter #18 Loss: 0.6656576312208539
iter #19 Loss: 0.6626808265139004
iter #20 Loss: 0.6607606564908464
iter #21 Loss: 0.6589171630779499
iter #22 Loss: 0.6579955389656996
iter #23 Loss: 0.6567567793775331
iter #24 Loss: 0.6558312401553701
iter #25 Loss: 0.6550229878973235
iter #26 Loss: 0.6538173174721941
iter #27 Loss: 0.6530954168381424
iter #28 Loss: 0.6521943405194937
iter #29 Loss: 0.65069085075

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.0644,  1.0421,  1.7337,  ...,  1.1874,  1.1629,  0.9372],
        [ 1.8074,  0.6461,  1.2161,  ...,  0.1621,  0.2567,  1.8258],
        [ 0.9084,  1.7954,  0.8384,  ..., -1.2373,  1.9695,  2.0541],
        ...,
        [ 0.9560,  2.5387,  1.8639,  ...,  1.4483,  0.8281,  0.2223],
        [ 0.3048,  0.6336,  0.9815,  ...,  0.8975,  1.2594,  0.9969],
        [ 1.2134,  1.0056,  1.1313,  ...,  0.4945,  2.0810,  1.3699]],
       device='cuda:0')
item_factors.weight tensor([[ 0.5624,  0.2631,  0.3133,  ...,  0.4449,  0.4555,  0.6236],
        [ 0.6967, -0.1422,  0.5169,  ...,  0.2638,  0.3142,  0.2842],
        [ 0.5184,  0.6106,  0.6313,  ...,  0.7618,  0.5348,  0.2878],
        ...,
        [ 0.3730,  0.3524,  0.3783,  ...,  0.3404,  0.3411,  0.3396],
        [ 0.4095,  0.4080,  0.4303,  ...,  0.4309,  0.3991,  0.4121],
        [ 0.4097,  0.4543,  0.4447,  ...,  0.4007,  0.4142,  0.4192]],
       device='cuda:0')


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Ghost (1990)
	 Clueless (1995)
	 Four Weddings and a Funeral (1994)
	 Cliffhanger (1993)
	 Casablanca (1942)
	 Wizard of Oz, The (1939)
	 Life Is Beautiful (La Vita è bella) (1997)
	 Dead Poets Society (1989)
	 Dave (1993)
	 Slumdog Millionaire (2008)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Schindler's List (1993)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Fugitive, The (1993)
	 Saving Private Ryan (1998)
Cluster #2
	 Willy Wonka & the Chocolate Factory (1971)
	 Breakfast Club, The (1985)
	 Kill Bill: Vol. 2 (2004)
	 Beetlejuice (1988)
	 Monty Python's Life of Brian (1979)
	 Total Recall (1990)
	 Ace Ventura: When Nature Calls (1995)
	 Spirited Away (Sen to Chihiro no kamikakushi) (2001)
	 When Harry Met Sally... (1989)
	 Blues Brothers, The (1980)
Cluster #3
	 True Lies (1994)
	 Mrs. D